# 设计Echart版本的ec数据读取形式

In [10]:
%%time
import geopandas as gpd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.interpolate import interp1d
import xesmf as xe
import redis
import pandas as pd
import datetime
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import os
import random
from io import BytesIO
import base64
from rasterio import features
from affine import Affine


class ec_data_point:
    def __init__(self,start_time,end_time):
        self.timelist = [0,2,4,6,8,10,12,14,16,18,20,22,24,25,
                         26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,
                        41,42,43,44,45,46,47,48,49,50,51,52]
        self.test_time = '2022041700'
        self.file_path = "/workspace/liyuan3970/Data/My_Git/" + self.test_time + "/" 
        self.lat_list = [27.532,27.532,27.532,27.532,27.532,27.532,27.532,27.532,27.532,27.532]
        self.lon_list = [121.932,121.932,121.932,121.932,121.932,121.932,121.932,121.932,121.932,121.932]
        self.name = ['洪家','玉环市','温岭市','椒江区','路桥区','黄岩区','仙居县','三门县','临海市','天台县']
        self.name_en = ['hongjia','yuhuan','wenling','jiaojiang','luqiao','huangyan','xianju','sanmen','linhai','tiantai']
        self.name_village = ['sanmen','linhai','xianju','tiantai','jiaojiang','wenling','yuhuan','luqiao','huangyan']
        self.cp,self.t2,self.lsp = self.read_data()
    # 外部函数
    def transform_from_latlon(self,lat, lon):
        lat = np.asarray(lat)
        lon = np.asarray(lon)
        trans = Affine.translation(lon[0], lat[0])
        scale = Affine.scale(lon[1] - lon[0], lat[1] - lat[0])
        return trans * scale    
    def rasterize(self,shapes, coords, latitude='lat', longitude='lon',fill=np.nan, **kwargs):
        transform = self.transform_from_latlon(coords[latitude], coords[longitude])
        out_shape = (len(coords[latitude]), len(coords[longitude]))
        raster = features.rasterize(shapes, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float, **kwargs)
        spatial_coords = {latitude: coords[latitude], longitude: coords[longitude]}
        return xr.DataArray(raster, coords=spatial_coords, dims=(latitude, longitude))
    def add_shape_coord_from_data_array(self,xr_da, shp_path, coord_name):   
        shp_gpd = gpd.read_file(shp_path)
        shapes = [(shape, n) for n, shape in enumerate(shp_gpd.geometry)]
        xr_da[coord_name] = self.rasterize(shapes, xr_da.coords, longitude='lon', latitude='lat')
        return xr_da
    # 数据分析
    def time_today(self):
        '''用于计算当前日期'''
        pass
    def regrid(self,data):
        # 插值
        ds_out = xr.Dataset(
            {   
                
                "lat": (["lat"], np.arange(27.0, 30, 0.05)),
                "lon": (["lon"], np.arange(120, 122.5, 0.05)),
            }
        )
        regridder = xe.Regridder(data, ds_out, "bilinear")
        dr_out = regridder(data)
        return dr_out
    def read_data(self):
        '''读取数据'''
        files = os.listdir(self.file_path)
        lsp_list = [] 
        cp_list = [] 
        t2_list = [] 
        for fileitem in self.timelist:
            f=xr.open_dataset(self.file_path +files[fileitem],decode_times=False)
            lsp = f.lsp.sel(lonS=slice(118,123),latS=slice(32,26))
            cp = f.cp.sel(lonS=slice(118,123),latS=slice(32,26))
            t2 = f.t2.sel(lonS=slice(118,123),latS=slice(32,26)) 
            lsp_list.append(lsp)  
            cp_list.append(cp)  
            t2_list.append(t2)
            del f,lsp,cp,t2  
        cp_all = xr.concat(cp_list,dim="time")
        cp_all = self.regrid(cp_all)
        t2_all = xr.concat(t2_list,dim="time")
        t2_all = self.regrid(t2_all)
        lsp_all = xr.concat(lsp_list,dim="time")
        lsp_all = self.regrid(lsp_all)
        return cp_all,t2_all,lsp_all
    def accum_data(self,list_data):
        '''处理累计降水'''
        out_list = []
        for i in range(len(list_data)):
            if i==0:
                out_list.append(0)
            elif i==1:
                out_list.append(round(list_data[i],1))
            else:
                out_list.append(round(list_data[i]-list_data[i-1],1))
        return out_list
    def rain_data(self,lat,lon,data):
        list_data= data.sel(lon=lon, lat=lat,method='nearest').to_pandas().tolist()
        out_list = self.accum_data(list_data)
        return out_list
    def plot_line(self,lat,lon):
        '''返回单点的降水气温曲线图'''
        cp_line = self.rain_data(lat,lon,self.cp)
        lsp_line = self.rain_data(lat,lon,self.lsp)
        t2_line = self.t2.sel(lon=lon, lat=lat,method='nearest').to_pandas().tolist()
        return cp_line,lsp_line,t2_line
    def village_data(self,data):
        data_xr = data
        shp_path = "shpfile/"
        shp_data = gpd.read_file("shpfile/xiangzhen/xiangzhen.shp", encoding='utf8')
        village_list = shp_data['NAME'].values
        county_list = shp_data['COUNTY'].values
        shp_da = self.add_shape_coord_from_data_array(data_xr, shp_path+"taizhou_village.shp", "country")
        data_dir = {
            "village":[],
            "county":[],
            "value":[]           
        }
        for i in range(len(village_list)):
            awash_da = shp_da.where(shp_da.country==i, other=np.nan)
            name = village_list[i]
            county = county_list[i]
            if np.isnan(awash_da.mean().values.tolist()):
                data_dir['value'].append(0)
            else:
                data_dir['value'].append(awash_da.mean().values.tolist())
            data_dir['village'].append(name)
            data_dir['county'].append(county)
        village = pd.DataFrame(data_dir)
        return village 
    def group_data(self,data):
        '''排序'''     
        rain = []
        village = []
        grouped = data.groupby('county')
        for i in grouped.size().index:
            village_list = []
            singel_rain = {
                "name":None,
                "value":None
            }
            county = grouped.get_group(i)
            sort_data = county.sort_values('value',ascending=False)[0:10]
            village_data = county.sort_values('value',ascending=False)[0:10]
            singel_rain['name'] = sort_data.county.tolist()[0]
            singel_rain['value']= round(county.value.mean(),2)
            rain.append(singel_rain)
            # 数据
            name = sort_data.county.tolist()[0]
            for j,k in zip(sort_data['village'],sort_data['value']):
                # print(j,k)
                singel_village ={}
                singel_village['value'] = round(k,2)
                singel_village['name'] = j            
                village_list.append(singel_village)
            village.append(village_list)
        return rain,village
    def comput_average(self,start_time,end_time):
        '''计算面雨量'''   
        data = { }
        if start_time==0:
            lsp = self.lsp[end_time,:,:] 
            comput = self.village_data(lsp)
            for i in range(len(self.name)):
                county = {
                    "line":{                
                        "lsp":None,
                        "cp":None,
                        "t2":None,
                        "time":None
                         }
                       }
                cp_line,lsp_line,t2_line = self.plot_line(self.lat_list[i],self.lon_list[i])
                county['line']['lsp'] = lsp_line
                county['line']['cp'] = cp_line
                county['line']['t2'] = t2_line 
                county['line']['time'] = self.timelist
                data[self.name_en[i]] = county
            rain,village= self.group_data(comput) 
            data["rain"] = rain
            data["village"] = village
        else:
            lsp1 = self.lsp[end_time,:,:] 
            lsp0 = self.lsp[start_time,:,:] 
            dif = lsp1 - lsp0 
            comput = self.village_data(dif)
            for i in range(len(self.name)):
                county = {
                    "line":{                
                        "lsp":None,
                        "cp":None,
                        "t2":None,
                        "time":None
                         }
                       }
                cp_line,lsp_line,t2_line = self.plot_line(self.lat_list[i],self.lon_list[i])
                county['line']['lsp'] = lsp_line
                county['line']['cp'] = cp_line
                county['line']['t2'] = t2_line 
                county['line']['time'] = self.timelist
                data[self.name_en[i]] = county
            rain,village= self.group_data(comput) 
            data["rain"] = rain
            data["village"] = village
        return data
    

        
        
        

#select_time,select_type,select_county = '2022041700','wind','仙居'
start_time,end_time = 0,24
ec_worker = ec_data_point(start_time,end_time) 
ec_worker.plot_line(27.532,121.932)

a = ec_worker.comput_average(start_time,end_time)


/opt/conda/lib/python3.7/site-packages/xarray/core/dataarray.py:784: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data


CPU times: user 2.49 s, sys: 76.3 ms, total: 2.57 s
Wall time: 3.28 s


In [6]:
a.keys()

dict_keys(['hongjia', 'yuhuan', 'wenling', 'jiaojiang', 'luqiao', 'huangyan', 'xianju', 'sanmen', 'linhai', 'tiantai', 'rain', 'village'])

In [20]:
a['village']

[[{'value': 1.76, 'name': '珠岙镇'},
  {'value': 1.34, 'name': '浦坝港镇'},
  {'value': 0.6, 'name': '健跳镇'},
  {'value': 0.54, 'name': '蛇蟠乡'},
  {'value': 0.33, 'name': '花桥镇'},
  {'value': 0.27, 'name': '亭旁镇'},
  {'value': 0.17, 'name': '海润街道'},
  {'value': 0.0, 'name': '海游街道'},
  {'value': 0.0, 'name': '沙柳街道'},
  {'value': 0.0, 'name': '横渡镇'}],
 [{'value': 3.1, 'name': '汇溪镇'},
  {'value': 1.18, 'name': '上盘镇'},
  {'value': 0.62, 'name': '大田街道'},
  {'value': 0.55, 'name': '河头镇'},
  {'value': 0.35, 'name': '江南街道'},
  {'value': 0.33, 'name': '小芝镇'},
  {'value': 0.29, 'name': '东塍镇'},
  {'value': 0.22, 'name': '杜桥镇'},
  {'value': 0.17, 'name': '汛桥镇'},
  {'value': 0.17, 'name': '尤溪镇'}],
 [{'value': 1.3, 'name': '福应街道'},
  {'value': 0.83, 'name': '田市镇'},
  {'value': 0.73, 'name': '皤滩乡'},
  {'value': 0.68, 'name': '南峰街道'},
  {'value': 0.42, 'name': '白塔镇'},
  {'value': 0.29, 'name': '安洲街道'},
  {'value': 0.27, 'name': '大战乡'},
  {'value': 0.27, 'name': '溪港乡'},
  {'value': 0.16, 'name': '淡竹乡'},
  {'value

In [ ]:
data = {
    "面雨量":[], 
    "临海市"：{
        "line":{
            "lsp":[],
            "cp":[],
            "t2":[],
            "time":[]
        },
        "village":{
            "rain":[],
            "line":[],
            "name":[]          
        }
    },
    
}

In [1]:
import geopandas as gpd


shp_path = "shpfile/xiangzhen/xiangzhen.shp"
shp_gpd = gpd.read_file(shp_path, encoding='utf8')
shp_gpd

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


,FEATUREGUI,FCODE,NAME,FSCALE,USOURCE,UPDATETIME,UPDATESTAT,SHAPE_Leng,SHAPE_Area,COUNTY,SHI,TYPE,CITYCODE,geometry
0,3dd51852d9dc44f5ab5cb0004bd766bf,6601000500,安洲街道,13,20190402基础库全量包,20190402,U,0.542087,0.004292,仙居县,台州市,4,None,"POLYGON ((120.68685 28.91842, 120.68606 28.914..."
1,8425d006aeb34b858d86120247526bef,6601000500,桃渚镇,13,20190402基础库全量包,20190402,U,1.018298,0.011036,临海市,台州市,4,None,"MULTIPOLYGON (((121.67986 28.79381, 121.67952 ..."
2,03b8f0e160754d608c5747b467d7e5c8,6601000500,蓬街镇,13,20190402基础库全量包,20190402,U,0.511900,0.006772,路桥区,台州市,4,None,"MULTIPOLYGON (((121.60227 28.55373, 121.60217 ..."
3,094b7e3bcc3144329512958c9d6fc8be,6601000500,杜桥镇,13,20190402基础库全量包,20190402,U,0.900278,0.016811,临海市,台州市,4,None,"POLYGON ((121.47616 28.82388, 121.47818 28.823..."
4,91d31aae17484169a101b846138f620c,6601000500,海门街道,13,2020年6月基础库政区数据,20200611,U,0.302182,0.002189,椒江区,台州市,4,None,"POLYGON ((121.50494 28.66132, 121.49950 28.658..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,5b8d3b8c1b044a2c886155549a6d753c,6601000500,大陈镇,13,2020年6月基础库政区数据,20200611,A,1.231533,0.001371,椒江区,台州市,4,None,"MULTIPOLYGON (((121.90887 28.39064, 121.90949 ..."
127,ab5f6c265fba43559d1cf7838a08a727,6601000500,城南镇,13,2020年6月基础库政区数据,20200611,A,1.055532,0.010046,温岭市,台州市,4,None,"MULTIPOLYGON (((121.43654 28.26580, 121.43835 ..."
128,566991a69fc24be5845186e4678c735a,6601000500,花桥镇,13,2020年6月基础库政区数据,20200611,A,0.569683,0.006595,三门县,台州市,4,None,"MULTIPOLYGON (((121.56132 28.93053, 121.56174 ..."
129,d19857af684e40f0ad5b03a2690f2765,6601000500,新桥镇,13,20190402基础库全量包,20190402,U,0.233467,0.001272,路桥区,台州市,4,None,"POLYGON ((121.42574 28.53229, 121.43003 28.525..."
